# Project 1

# Fake-News Cleaning+Word2Vec+LSTM (99% Accuracy)

dataset in kaggle in:

https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

https://www.kaggle.com/atishadhikari/fake-news-cleaning-word2vec-Istm-99-accuracy

divided to 2 files:
-Fake.csv
-True.csv


In [ ]:
import warnings 
warnings.filterwarnings ('ignore')
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk 
import re
from wordcloud 
import WordCloud   #pip install wordcloud 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, accuracy_score
fake = pd.read_csv ("Fake.csv")
print (fake.head ( ))

# Counting by Subjects
for key, count in fake.subject.value_counts ().iteritems ():
    print(f"{key}:\t{count}")
# Getting Total Rows
print (f"Total Records: \t{fake.shape [0]}")
pit.figure(figsize=(8,5))
sns.countplot ("subject", data=fake)
plt.show()

In [ ]:
#Word Cloud
# will show the most words freq in text
text = ""
for news in fake.text.values:
    text += f" {news}"
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow (wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout (pad=0)
plt.show()
del text


In [ ]:
real = pd.read_csv ("True.csv")
print (real.head ())
#First Creating list of index that do not have publication part
unknown_publishers = []
for index, row in enumerate (real.text.values):
    try:
        record = row.split(" -", maxsplit=1)
        #if no text part is present, following will give error 
        record [1]
        #if len of piblication part is greater than 260 
        #following will give error, ensuring no text having "_" in between is counted
        assert(len(record [0]) ‹ 260)
    except:
        unknown_publishers.append(index)
#Thus we have list of indices where publisher is not mentioned 
#lets check
print (real. iloc [unknown_publishers].text)
#true, they do not have text like "WASHINGTON (Reuters)"


print (real. iloc [8970])
#уер empty
#will remove this soon
# Seperating Publication info, from actual text
publisher = []
tmp_text = []
for index, row in enumerate (real. text. values) :
    if index in unknown_publishers:
        # Add unknown of publisher not mentioned
        tmp_text.append(row)
        publisher .append("Unknown")
        continue
    record = row.split(" -", maxsplit=1)
    publisher.append(record[0])
    tmp text. append (record[1])
# Replace existing text column with new text
# add seperate column for publication info
real["publisher"] = publisher
real["text"] = tmp text


del publisher, tmp_text, record, unknown_publishers 
print ( real. head())
#checking for rows with empty text like row: 8970
[print("empty text:",index) for index, text in enumerate (real.text.values) if str(text).strip() == ''] 
#seems only one :)
#empty text: 8970
#dropping this record
real = real. drop (8970, axis=0)
# checking for the same in fake news
empty_fake_index = [index for index, text in enumerate (fake.text.values) if str(text).strip() == '']
print (f"No of empty rows: {len (empty_fake_index)}")
fake. iloc[empty_fake_index].tail()
#No of empty rows: 630
#checking for rows with empty text like row: 8970
[print("empty text:", index) for index, text in enumerate (real.text.values) if str(text) .strip() == "']
#seems only one :)
 
sns.countplot(x="subject", data=real) 
plt.show()

In [ ]:
#WordCloud For Real News
text = ''
for news in real.text.values:
    text += f" {news}"
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(str (text))
fig = pit.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis ('off')
plt.tight_ layout (pad=0)
plt.show()
del text


# Adding class Information
real["class"] = 1
fake ["class"] = 0
#Combining Title and Text
real ["text"] = real["title" ]+ " " + real ["text"]
fake["text"] = fake["title"] + " " + fake["text"]
# Subject is diffrent for real and fake thus dropping it
# Also dropping Date, title and Publication Info of real
real = real.drop (["subject", "date", "title", "publisher"], axis=1)
fake = fake.drop(["subject", "date", "title"], axis=1)
#Combining both into new dataframe
data = real.append(fake, ignore_index=True)
del real, fake


# Download following if not downloaded in local machine
# nltk.download('stopwords")
# nLtk. download ( 'punkt")

#Removing StopWords, Punctuations and single-character words
y = data["class"].values
#Converting X to format acceptable by gensim, removing and punctuation stopwords in the process
X = []
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = nltk.tokenize. RegexpTokenizer (r'\w+')
for par in data["text"].values:
    tmp = []
    sentences = nltk.sent tokenize(par)
    for sent in sentences:
        sent = sent. lower ()
        tokens = tokenizer.tokenize (sent)
        filtered _words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
        tmp.extend(filtered_words)
    X. append (tmp)

    
import gensim #pip install gensim
#Dimension of vectors we are generating
EMBEDDING_DIM = 100
#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences=X, size=EMBEDDING_DIM, window=5, min_count=1)
#vocab size 
len (w2v_model.wv.vocab)
#We have now represented each of 122248 words by a 100dim vector.
#see a sample vector for random word, lets say Corona 
print (w2v_model ["corona"])


In [ ]:
print ( w2v_model.wv.most_similar("iran"))
print ( w2v_model.wv.most_similar("fbi"))
print ( w2v_model.wv.most_similar("facebook"))
print ( w2v_model.wv.most_similar("computer"))
#Feeding US Presidents
print ( w2v_model.wv.most_similar(positive=["trump", "obama", "clinton"]))
#First was Bush


# Tokenizing Text -> Repsesenting each word by a number
# Mapping of orginal word to number is preserved in word_index property of tokenizer
#Tokenized applies basic processing Like changing it yo Lower case, explicitely setting that as False
tokenizer = Tokenizer()
tokenizer.fit_on_texts (X)
X = tokenizer.texts_to_sequences (X)
# lets check the first 10 words of first news
#every word has been represented with a number 
print ( X[0][:10])
#Lets check few word to numerical replesentation
#Mapping is preserved in dictionary -> word_ index property of instance
word_index = tokenizer.word_index
for word, num in word_index.items ():
    print (f"{word} -> {num}")
    if num == 10:
        break

# For determining size of input.
# Making histogram for no of words in news shows that most news
# Lets keep each news small and truncate all news to 700 while t
pit.hist([len(x) for × in X], bins=500)
plt.show()
# Its heavily skewed. There are news with 5000 words? Lets trunc
nos = np.array([len(x) for × in X])
len (nos [nos ‹ 7001])
# Out of 48k news, 44k have less than 700 words
#Lets keep all news to 700, add padding to news with less than + +aa
maxlen = 700

#Making all news of size maxlen defined above
X = pad_sequences (X, maxlen=maxlen)
#all news has 700 words (in numerical form now). If they had less words, they have been padded with o
# 0 is not associated to any word, as mapping of words started from 1
# o will also be used later, if unknows word is encountered in test set
len (X[0])
# Adding 1 because of reserved 0 index
# Embedding Layer creates one more vector for "UNKNOWN" words, or padded words (Os). This Vector is filled with zeros
# Thus our vocab size inceeases by 1
vocab_size = len(tokenizer.word_index) + 1
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len (vocab) + 1
    # define weight matrix dimensions with all o
    weight_matrix = np.zeros ((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items ():
        weight_matrix[i] = model [word]
    return weight_matrix

#Getting embedding vectors from wordzvec and usings it as weights of non-trainable keras embedding layer
embedding_vectors = get_weight_matrix(w2v_model, word_index)
#Defining Neural Network
model = Sequential ()
#Non-trainable embedding Layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128))
model. add(Dense (1, activation='sigmoid'))
model. compile (optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
del embedding_vectors 
print ( model.summary ())
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)
model.fit (X_train, y_train, validation_split=0.3, epochs=6)

#Prediction is in probability of news being real, so converting into classes
# Class 0 (Fake) if predicted prob ‹ 0.5, else class 1 (Real) 
y_pred = (model.predict (X_test) ›= 0.5).astype("int") 
accuracy_score (y_test, y_pred)
print (classification_report(y_test, y_pred))

# Project 2

# NLP for fake news detection

dataset in kaggle in:

https://www.kaggle.com/vibinmarish/nlp-for-fake-news-detection


In [ ]:
import numpy as np # Linear algebra
import pandas as pd # data processing, CSV file I/0 (e.g. pd.read_csv)
tn = pd.read_csv("True.csv", nrows=4000)
fn = pd.read_cs®("Fake.csv", nrows=4000) 
print (fn.head ())
tn[ 'class'] = 1
fn['class'] = 0
print (tn.head())
df = pd.concat ([tn. head (4000), fn. head (4000)], ignore_index = True)
print (df)
f = df.sample(frac=1).reset_index(drop=True)
print (df)
df['News'] = df['title'] + " " + df['text'] + " " + df['subject' ]
print (df)


import re 
import nltk
from nltk.corpus import stopwords 
from nitk.stem import WordNetLemmatizer
TL = WordNetLemmatizer ()
corpus = []
for i in range (0,8000):
    line = re.sub('[^a-zA-Z]', " ", df[ 'News '][i])
    line = line.lower ()
    line = line.split()
    line = [TL.lemmatize(word) for word in line if not word in set(stopwords.words('english'))]
    line = " ".join(line)
    corpus.append (line)
from sklearn. feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray ()
y = df['class'].values


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
from sklearn.naive_bayes import GaussianNB 
model=GaussianNB()
final=model.fit (X_train,y_train)
print("Testing score", final.score(X_test,y_test))

import seaborn as sb 
from sklearn.metrics import confusion matrix 
from matplotlib import pyplot as plt 
svm_predicted=final.predict (X test)
svm_confuse=confusion_matrix(y_test, svm_predicted) 
df_cm=pd.DataFrame (svm_ confuse)

plt.figure(figsize=(5.5,4))
sb. heatmap (df_cm, annot=True, fmt= 'g') 
plt.title("Confusion Matrix Heatmap") 
plt.xlabel("True Label") 
plt.ylabel("Predicted Label") 
plt.show()

from sklearn.metrics import classification_report
print("classification Report")
print(classification_report (y_test,sm_predicted))

from sklearn.neighbors import KNeighborsClassifier 
model=KNeighborsClassifier (n_neighbors=2)
final=model.fit (X_train,y_train)
print("Testing score", final.score(X_test,y_test))
import seaborn as sb 
from sklearn.metrics import confusion_ matrix 
from matplotlib import pyplot as plt
knn_predicted=final.predict(X_test)
knn_confuse=confusion_matrix(y_test, kn_predicted) 
df_cm=pd.DataFrame (knn_confuse)

pit.figure(figsize= (5.5,4))
sb.heatmap (df_cm, annot=True, fmt= 'g' )  
plt.title("Confusion Matrix Heatmap") 
plt.xlabel("True Label") 
plt.ylabel("Predicted Label") 
plt.show()

from sklearn.metrics import classification_report
print("Classification Report")
print(classification_report (y_test,knn_predicted))

# Project 3

# NB and RF models (99% accuracy)

dataset in kaggle in:

https://www.kaggle.com/rahulvv/nb-and-rf-models-99-accuracy

In [ ]:
import numpy as np # Linear algebra
import pandas as pd # data processing, CSV file I/0 (e.g. pd. read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 
import re 
import nltk
from nitk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.stem.wordnet import WordNetLemmatizer 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 
import string
from sklearn.model selection import train_test_split 
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score 
from sklearn.naive bayes import MultinomiaINB 
from sklearn. feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.pipeline import Pipeline 
import xgboost as xgb 
from sklearn.ensemble import RandomForestClassifier

seed = 4353
true = pd.read _csv('True.csv")
fake = pd.read_csv('Fake.csv')
# Introducing new column in both dataframes
true ['impression']=1
fake[ 'impression'1=0
# Concatenating them using pandas concatenate to form a single dataframe
data_raw = pd. concat ([true, fake], axis=0)
# Combining title and text to obtain a single string
# dropping title and
data_raw['fulltext'] = data_raw.title + ' ' + data_raw.text
data_raw.drop(['title', 'text'], axis=1, inplace=True)
# Extracting a new dataframe using features fulltext and impression
data = data_raw[[ 'fulltext', 'impression']]
data = data.reset_index ()
data.drop(['index'], axis=1, inplace=True)
# Check for missing values

print ( data.isnull ().sum())
print('The dataset contans {} rows and {} columns'.format (data.shape [0], data.shape [1]))
# Word extraction from true and fake texts
true_text = data [data.impression == 1]['fulltext']
fake_text = data[data.impression == 0]['fulltext']
fake_text = fake_text.reset_index().drop(['index'], axis=1)

# Function to extract major words from true and fake news
def wordcloud_words(X_data_full):
# function for removing punctuations
    def remove_punct (X_data_func):
        string1 = X_data_func. lower ()
        translation_table = dict.fromkeys (map(ord, string.punctuation), " ")
        string2 = striag1.translate (translation_table)
        return string2
     
    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct (X_data_full[i])
        X_data_full_clear_punct.append(test_data)
     
    # function to remove stopwords
    def remove_stopwords (X_data_func) :
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words("english")) + r') \b|s*')
        string2 = pattern.sub(' ', X_data_func)
        return string2
     
    X_data_full_clear_stopwords = []
    for i in range(len (X_data_full)):
        test_data = remove_stopwords (X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)

    # function for tokenizing
    def tokenize_words (X_data_func):
        words = nitk.word_tokenize(X_data_func)
        return words
    X_data_full_tokenized_words = []
    for i in range (len (X_data_full)):
        test_data = tokenize_words (X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        Output
    # function for lemmatizing
    lemmatizer = WordNetLemmatizer ()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
     X_data_full_lemmatized_words = []
     for i in range(len (X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        × data full lemmatized_words.append(test_data)
    return X_data_full_lemmatized_words

true_words = wordcloud_words (true_text)
fake_words = wordcloud_words (fake_text.fulltext)
def plot wordcloud (text):
    wordcloud = WordCloud (background_color='black', max_words=3000, width=1600, height=800).generate(text)

    plt.clf()
    plt.imshow(wordcloud, interpolation='bilinear') 
     plt.axis('off') 
     plt.show()
plt.figure(figsize=(20, 18)) 
plot_wordcloud(' '.join(true_words)) 
plt.show() 
plt.figure(figsize=(20, 18)) 
plot_wordcloud(' '.join(fake_words) ) 
plt.show()

# Data preparation
X _data = data[ 'fulltext ']
y_data = data. impression
X_data = X_data. astype(str)
# Function to retrieve processed words
def final(X_data_full):
# function for removing punctuations
    def remove_punct (X_data_func):
        string1 = X_data_func.lower ()
        translation_table = dict.fromkeys (map(ord, string-punctuation), " ")
        string2 = string1.translate (translation_table)
        return string2
     X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)
        # function to remove stopwords
    def remove_stopwords (X_data_func):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r') \b\s*')
        string2 = pattern. sub(' ', X_data_func)
        return string2

    X_data_full_clear_stopwords = []
    for i in range(len(X_data_ full)):
        test_data = remove_stopwords (X_data_ful1[i])
        X_data_full_clear_stopwords.append(test_data)
        # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words
    X_data_full_tokenized_words = []
    for i in range(len (X_data_full)):
        test_data = tokenize_words (X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        # function for lemmatizing
     lemmatizer = WordNetLemmatizer ()
     def lemmatize_words (X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
     X_data_full_lemmatized _words = []
     for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)

        # creating the bag of words model
        cV = CountVectorizer (max_features=1000)
        X_data_full_vector = cv.fit_transform(X_data_full_lemmatized_words).toarray ()
        tfidf = TfidfTransformer )
        X_data_full_tfidf = tfidf.fit_transform(X_data_full_vector).toarray()
        return X_data_full_tfidf
# Setting the function with parameters
data X = final (X data)
# Preparing training and testing data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_X, y_data, test_size=0.25, random_state=seed)

# Instatiation, fitting and prediction
MNB = MultinomialNB ()
MNB. fit (X_train, y_train)
predictions = MNB.predict(X test)
# Model evaluation
print(classification_report (_test, predictions))
print(confusion_matrix(y_test, predictions))
# Instatiation, fitting and prediction
rfc=RandomForestClassifier(n_estimators= 10, random_state= seed) 
rfc.fit(X_train, y_train)
predictions = rfc.predict(X_test)
# Model evaluation
print (classification_report (y_test, predictions))
print(confusion_matrix(y_test, predictions))